In [ ]:
def classification_bidirectional_lstm(text):
    ############### ▼패키지 임포트▼ ###############
    import numpy as np
    import pickle
    import tensorflow as tf
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding
    from tensorflow.keras.optimizers import Adam
    from konlpy.tag import Mecab
    ############### ▲패키지 임포트▲ ###############
    
    # 모델링에 사용한 토크나이저의 내부 단어장을 불러옵니다 
    pickle_path = '분류기/Save_Vocab/'
    with open(pickle_path + 'src_tokenizer.p', 'rb') as f:
        src_tokenizer = pickle.load(f)
    with open(pickle_path + 'tar_tokenizer.p', 'rb') as f:
        tar_tokenizer = pickle.load(f)
    
    # 단어장의 '인덱스-단어, 인덱스-태그, 인덱스-단어 갯수, 인덱스-태그 갯수' 를 추출하여 변수로 받습니다
    index_to_word = src_tokenizer.index_word
    index_to_tag = tar_tokenizer.index_word
    vocab_size = len(src_tokenizer.word_index) + 1
    tag_size = len(tar_tokenizer.word_index) + 1

    # 기학습한 모델을 생성합니다
    max_len = 200
    embedding_dim = 128
    hidden_units = 128

    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, mask_zero= True))
    model.add(Bidirectional(LSTM(hidden_units, return_sequences= True)))
    model.add(TimeDistributed(Dense(tag_size, activation= ('softmax'))))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
    
    # 모델의 가중치를 불러옵니다
    saved_path = '분류기/Save_Model/bidirectional_lstm_(final)_Project_tags/'
    model.load_weights(saved_path + 'bidirectional_lstm_(final)_Project_tags')
    
    # '형태소 분석기 프로젝트 "은전한닢"' 의 mecab 을 불러와 객체화합니다 
    mecab = Mecab(dicpath = r"분류기/BiLSTM_Model/mecab/mecab-ko-dic")
    # 입력 text를 받아 형태소 분리를 합니다
    morpheme = mecab.morphs(text)

    # 분리된 형태소를 모델의 예측을 통해 형태소 분리된 단어들과 예측된 태그들을 최종 출력값으로 return 합니다
    result_sen = []
    result_tag = []

    for i in range(len(morpheme)):
        sample = src_tokenizer.texts_to_sequences(morpheme)
        sample = pad_sequences(sample, padding= 'post', maxlen=max_len)


        y_predicted = model.predict(np.array([sample[i]]))
        y_predicted = np.argmax(y_predicted, axis= -1)

        for word, pred in zip(sample[i], y_predicted[0]):
            if word != 0:
                result_sen.append(index_to_word[word])
                result_tag.append(index_to_tag[pred].upper())
    
    return result_sen, result_tag